In [1]:
"""
Conversion steps:
* Get a list of all notes by filtering on files with extension json
* Read each file
* Create a note with title, text, tag `#journey`
* Paste remaining keys as metadata at the end of the script
* If tags are present, create them as bear tags
* Call bear's x-callback-url create-note to create the note
"""

"\nConversion steps:\n* Get a list of all notes by filtering on files with extension json\n* Read each file\n* Create a note with title, text, tag `#journey`\n* Paste remaining keys as metadata at the end of the script\n* If tags are present, create them as bear tags\n* Call bear's x-callback-url create-note to create the note\n"

In [11]:
import os
import sys
import shutil

import json
import logging
from pathlib import Path
import re

import html2markdown
import requests

logging.basicConfig(level=logging.ERROR)
journey_notes_dir = Path.cwd() / 'raw' / 'journey-backup'
Path.mkdir(Path.cwd() / 'final', exist_ok=True)
bear_notes_dir = Path.cwd() / 'final'

# note_fp = list(journey_notes_dir.glob('*.json'))
note_fp = [journey_notes_dir / '1524195338279-3fb6d72698a2ab50.json']
for fp in note_fp:
    with open(fp, 'r') as fh:
        input_note = json.load(fh)
        
    # journey does not store an explicit title. So take the first line as the title  
        title_pattern = r'(.*?)[\n\-\.](.*)'
        try:
            match = re.search(title_pattern, input_note['text'], re.DOTALL)
            if match:
                # in case the body is None, it will be defaulted to ''
                title, body = match.groups('')
            else:
                title = input_note['text']
                body = ''
                logging.debug('note {} contains a one-liner text'.format(fp.name))
#                 raise Exception('No match found for {}'.format(fp.name))
            
            logging.debug('filename: {}\n Original title: {} \n Original body: {}'.format(fp.name, title, body))
            
            title_in_md = html2markdown.convert(title).strip()
            body_in_md = html2markdown.convert(body).strip()
            logging.debug('filename: {}\n Converted title: {} \n Converted body: {}'.format(fp.name, title_in_md, body_in_md))
        except Exception as e:
            logging.exception(e)
        
        # add media
        media_list = []
        if input_note['photos']:
            media_list = input_note['photos']
        
        output_note_fp = bear_notes_dir / '{}.md'.format(input_note['id'])
        with open(output_note_fp, 'w') as fh:
            fh.write('\n'.join([title_in_md, body_in_md]))
            fh.write('\n\n')
            if media_list:
                fh.write('\n'.join(['![]({})'.format(media) for media in media_list]))
                for media in media_list:
                    shutil.copy(journey_notes_dir / media, bear_notes_dir / media)
            
            
    

In [3]:
type(input_note)

dict

In [4]:
?Path.co

<class 'pathlib.PosixPath'>
